#### 다양한 모델을 결합한 앙상블
- 앙상블
- 다수결 투표 앙상블 (Voting)
- 배깅(Bagging)
- 에이다부스트(AdaBoost)
- 그레이던트 부스팅& XGBoost
- 모델 성능 평가 및 비교


- 단일모델의 한계
    - 과대적합(Overfitting) : 학습데이터에 너무 맞춰져있어서 새로운 데이터에 대한 성능저하 
    - 과소적합(Underfitting) : 모델이 너무 단순해서 제대로 학습이 안됨
    - 높은 분산(High varience) : 학습 데이터가 조금만 바뀌어도 모델이 크게 달라지는 현상
    - 높은 편향(High Bias) : 모델이 진짜 패턴을 포착하지 못함
- 앙상블(Ensemble) : 집단지성
    - 배깅 : 같은 알고리즘이지만 서로 다른 데이터셋(부트스트랩)  ex.RandomForest, 주요기능: 분산감소
    - 부스팅 : 순차적으로 약한 학습기를 강화 ex.AdaBoost, XGBoost, 주요기능: 편향감소
    - 스태킹 : 서로 다른 알고리즘을 메타 모델로 학습, 주요기능: 일반화의 성능 

- 약한 학습기 : 최소한의 모델
- 강한 학습기 : 성능이 뛰어난 모델

- Voting
    - Hard Voting : 다수결 투표
        A:고양이
        B:강아지
        C:고양이
    - Soft Voting : 가중치 투표 - 확률을 보고 결정, 보통 Hard Voting 보다 성능이 좋음
        고양이, 강아지
        A:(0.9,0.1)
        B:(0.4,0.6)
        C:(0.6,0.2)
        평균확률 : ,
- Bagging(Bagging - Bootstrap Aggregation)
    - Bootstrap : 원본 데이터 1000개면 중복을 허용해서 1000 뽑아 훈련세트 1개를 만듦...
    - Aggregation
        - 순차적으로 학습을 하고 난 후에 여러개의 모델들을 투표(Voting)해서 최종 결론
    - 대표모델은 RandomForest
    - 효과 : 과적합을 방지 즉, 모델의 분산을 줄여줌
- Boosting
    - 모델들이 순서대로 학습, 앞 모델의 실수를 뒷 모델이 보완